In [1]:
import sqlite3
import logging
import pandas as pd
import numpy as np
import plotly.express as px
from pathlib import Path
from sklearn.preprocessing import LabelEncoder

# Configuration
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [2]:
class DatabaseManager:
    """Gerencia conexões e operações com bancos de dados."""

    @staticmethod
    def get_database_connection():
        """Gerenciador de contexto para conexões de banco de dados com limpeza adequada de recursos."""

        # Pega a pasta raiz do projeto
        notebook_dir = Path.cwd()
        project_root = (
            notebook_dir.parent
            if notebook_dir.name == "notebooks"
            else notebook_dir
        )

        # Conexão com SQLite
        db_path = project_root / "data" / "db" / "database.db"

        # Garantir que o diretório exista
        db_dir = db_path.parent
        if not db_dir.exists():
            db_dir.mkdir(parents=True)

        conn = None
        try:
            conn = sqlite3.connect(db_path)
            yield conn
        except Exception as e:
            logger.error(f"Erro ao conectar com o banco de dados: {e}")
            raise
        finally:
            if conn:
                conn.close()

    def execute_query(self, query, params=None):
        """Executa uma consulta no SQLite."""
        try:
            cursor = self.conn.cursor()
            cursor.execute(query, params or ())
            result = cursor.fetchall()
            return result
        except Exception as e:
            self.logger.error(f"Falha ao executar consulta: {e}")
            return None

    def disconnect(self):
        """Fecha todas as conexões."""
        if self.conn:
            self.conn.close()


In [3]:
# Configurar pandas para exibir números sem notação científica
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.precision", 2)

# Carregar dados do Banco
db_manager = DatabaseManager()
for conn in db_manager.get_database_connection():
    query = "SELECT * FROM vw_aai"
    cursor = conn.cursor()
    cursor.execute(query)
    columns = [column[0] for column in cursor.description]
    df = pd.DataFrame.from_records(cursor.fetchall(), columns=columns)

if df["data_referencia"].dtype == "object":
    df["data_referencia"] = pd.to_datetime(
        df["data_referencia"], format="%Y-%m-%d"
    )

le = LabelEncoder()
for column in df.select_dtypes(include=["object", "datetime64"]).columns:
    df[column] = le.fit_transform(df[column])

# Criar cópia mascarada para visualização (valores sensíveis ocultos)
df_display = df.copy()


def mask_value(x):
    """Mascara valores numéricos mantendo apenas os 2 primeiros dígitos
    Aplica também para casas decimais."""
    if pd.isna(x) or x == 0:
        return x

    # Se for integer, manter apenas o primeiro dígito
    if isinstance(x, int):
        return str(x)[0] + "*"

    # Formatar com 2 casas decimais
    formatted = f"{abs(x):.2f}"

    # Separar parte inteira e decimal
    if "." in formatted:
        integer_part, decimal_part = formatted.split(".")
    else:
        integer_part = formatted
        decimal_part = ""

    # Mascarar parte inteira se tiver mais de 2 dígitos
    if len(integer_part) > 2:
        masked_int = integer_part[:2] + "*" * (len(integer_part) - 2)
    else:
        masked_int = integer_part

    # Mascarar parte decimal
    masked_dec = "**" if decimal_part else ""

    # Reconstruir valor
    if masked_dec:
        result = f"{masked_int}.{masked_dec}"
    else:
        result = masked_int

    return result if x >= 0 else f"-{result}"


# Aplicar máscara apenas em colunas numéricas (exceto data_referencia e codigo_assessor)
for column in df_display.select_dtypes(include=["float64", "int64"]).columns:
    if column not in ["data_referencia", "codigo_assessor"]:
        df_display[column] = df[column].apply(mask_value)

df_display.head()


,data_referencia,codigo_assessor,net_total,net_renda_fixa,net_fundos_imobiliarios,net_renda_variavel,net_fundos,net_financeiro,net_previdencia,net_outros,...,receita_aluguel_total,receita_complemento_total,clientes_novos,clientes_perdidos,clientes_pj,clientes_pf,volume_operado_rf,volume_operado_rv,saldo_cliente_total,saldo_cliente_medio
0,0,0,72******.**,42******.**,32*****.**,76*****.**,16******.**,27****.**,73****.**,18*****.**,...,27*.**,0,0,0,1*,1*,41*****.**,55*****.**,12****.**,10**.**
1,0,1,93******.**,51******.**,25*****.**,15******.**,19******.**,61****.**,30*****.**,94****.**,...,40*.**,0,0,0,1*,8*,23*****.**,27*****.**,63****.**,73**.**
2,0,2,26*******.**,10*******.**,62*****.**,49******.**,92******.**,14*****.**,70*****.**,55*****.**,...,43*.**,0,0,1*,2*,1*,49*****.**,68******.**,24*****.**,20***.**
3,0,3,11******.**,31*****.**,37*****.**,11*****.**,35*****.**,27***.**,44****.**,29**.**,...,67.**,0,0,0,2*,7*,43****.**,59*****.**,27***.**,38*.**
4,0,4,75*****.**,36*****.**,23*****.**,37****.**,75****.**,44***.**,28****.**,40**.**,...,1.**,0,0,1*,1*,1*,50***.**,48****.**,76***.**,40*.**


In [4]:
# Criar visualização da decomposição da receita com escala logarítmica
df_plot = df.copy()
df_plot = df_plot.sort_values(by="receita_bruta_total", ascending=False).head(
    50
)
df_plot["codigo_assessor"] = df_plot["codigo_assessor"].astype(str)

# Selecionar apenas colunas de receita relevantes, excluindo receita_liquida_total
receita_columns = [
    "receita_bovespa_total",
    "receita_futuros_total",
    "receita_rf_bancarios_total",
    "receita_rf_privados_total",
    "receita_rf_publicos_total",
    "receita_aluguel_total",
    "receita_complemento_total",
]

# Criar DataFrame no formato longo para plotly
df_long = df_plot.melt(
    id_vars=["codigo_assessor"],
    value_vars=receita_columns,
    var_name="Categoria",
    value_name="Valor",
)

# Aplicar transformação logarítmica para melhor visualização de valores pequenos
# Adicionar 1 para evitar log(0)
df_long["Valor_Log"] = df_long["Valor"].apply(
    lambda x: np.log10(x + 1) if x > 0 else 0
)

# Criar coluna mascarada com valores originais
df_long["Valor_Masked"] = df_long["Valor"].apply(mask_value)

# Mapear nomes mais amigáveis
category_names = {
    "receita_bovespa_total": "BOVESPA",
    "receita_futuros_total": "FUTUROS",
    "receita_rf_bancarios_total": "RF BANCÁRIOS",
    "receita_rf_privados_total": "RF PRIVADOS",
    "receita_rf_publicos_total": "RF PÚBLICOS",
    "receita_aluguel_total": "ALUGUEL",
    "receita_complemento_total": "COMPLEMENTO",
}
df_long["Categoria"] = df_long["Categoria"].map(category_names)

# Criar gráfico de barras empilhadas horizontal com escala logarítmica
fig = px.bar(
    df_long,
    x="Valor_Log",  # Usar valor logarítmico para visualização
    y="codigo_assessor",
    color="Categoria",
    title="Decomposição da Receita por Código de Assessor (Top 50)",
    labels={
        "codigo_assessor": "Código do Assessor",
        "Valor_Log": "Receita",
    },
    custom_data=["Valor_Masked", "Categoria"],
    orientation="h",
)

# Atualizar layout para melhor visualização
fig.update_traces(
    hovertemplate="<b>Assessor:</b> %{y}<br>"
    + "<b>Categoria:</b> %{customdata[1]}<br>"
    + "<b>Valor:</b> %{customdata[0]}<br>"
    + "<extra></extra>"
)
fig.update_layout(
    xaxis_title="Receita",
    xaxis=dict(showticklabels=False),  # Esconder valores no eixo X
    yaxis=dict(categoryorder="total ascending", showticklabels=False),
    yaxis_title="",
    height=800,
    barmode="stack",
    showlegend=True,
    plot_bgcolor="white",
    paper_bgcolor="black",
    title_font_color="white",
    legend=dict(
        title="Categoria",
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02,
        font=dict(color="white"),
    ),
)

fig.update_xaxes(title_font=dict(color="white"))

fig.show()


In [5]:
# Criar visualização da decomposição da custódia com escala logarítmica
df_plot = df.copy()
df_plot = df_plot.sort_values(by="net_total", ascending=False).head(50)
df_plot["codigo_assessor"] = df_plot["codigo_assessor"].astype(str)

# Selecionar apenas colunas de custódia (net) relevantes, excluindo net_total
net_columns = [
    "net_renda_fixa",
    "net_fundos_imobiliarios",
    "net_renda_variavel",
    "net_fundos",
    "net_financeiro",
    "net_previdencia",
    "net_outros",
]

# Criar DataFrame no formato longo para plotly
df_long = df_plot.melt(
    id_vars=["codigo_assessor"],
    value_vars=net_columns,
    var_name="Categoria",
    value_name="Valor",
)

# Aplicar transformação logarítmica para melhor visualização de valores pequenos
# Adicionar 1 para evitar log(0)
df_long["Valor_Log"] = df_long["Valor"].apply(
    lambda x: np.log10(x + 1) if x > 0 else 0
)

# Criar coluna mascarada com valores originais
df_long["Valor_Masked"] = df_long["Valor"].apply(mask_value)

# Mapear nomes mais amigáveis
category_names = {
    "net_renda_fixa": "RF",
    "net_fundos_imobiliarios": "FII",
    "net_renda_variavel": "RV",
    "net_fundos": "FUNDOS",
    "net_financeiro": "FINANCEIRO",
    "net_previdencia": "PREVIDÊNCIA",
    "net_outros": "OUTROS",
}
df_long["Categoria"] = df_long["Categoria"].map(category_names)

# Criar gráfico de barras empilhadas horizontal com escala logarítmica
fig = px.bar(
    df_long,
    x="Valor_Log",  # Usar valor logarítmico para visualização
    y="codigo_assessor",
    color="Categoria",
    title="Decomposição da Custódia por Código de Assessor (Top 50)",
    labels={
        "codigo_assessor": "Código do Assessor",
        "Valor_Log": "Custódia",
    },
    custom_data=["Valor_Masked", "Categoria"],
    orientation="h",
)

# Atualizar layout para melhor visualização
fig.update_traces(
    hovertemplate="<b>Assessor:</b> %{y}<br>"
    + "<b>Categoria:</b> %{customdata[1]}<br>"
    + "<b>Valor:</b> %{customdata[0]}<br>"
    + "<extra></extra>"
)
fig.update_layout(
    xaxis_title="Custódia",
    xaxis=dict(showticklabels=False),  # Esconder valores no eixo X
    yaxis=dict(categoryorder="total ascending", showticklabels=False),
    yaxis_title="",
    height=800,
    plot_bgcolor="white",
    paper_bgcolor="black",
    title_font_color="white",
    barmode="stack",
    showlegend=True,
    legend=dict(
        title="Categoria",
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02,
        font=dict(color="white"),
    ),
)

fig.update_xaxes(title_font=dict(color="white"))

fig.show()


In [6]:
# Criar visualização da decomposição da captação com escala logarítmica
df_plot = df.copy()
df_plot = df_plot.sort_values(
    by="captacao_liquida_total", ascending=False
).head(50)
df_plot["codigo_assessor"] = df_plot["codigo_assessor"].astype(str)

# Selecionar apenas colunas de captação relevantes, excluindo captacao_liquida_total
cap_columns = [
    "captacao_ted_total",
    "captacao_st_total",
    "captacao_ota_total",
    "captacao_rf_total",
    "captacao_td_total",
    "captacao_prev_total",
]

# Criar DataFrame no formato longo para plotly
df_long = df_plot.melt(
    id_vars=["codigo_assessor"],
    value_vars=cap_columns,
    var_name="Categoria",
    value_name="Valor",
)

# Aplicar transformação logarítmica para melhor visualização de valores pequenos
# Adicionar 1 para evitar log(0)
df_long["Valor_Log"] = df_long["Valor"].apply(
    lambda x: np.log10(x + 1) if x > 0 else 0
)

# Criar coluna mascarada com valores originais
df_long["Valor_Masked"] = df_long["Valor"].apply(mask_value)

# Mapear nomes mais amigáveis
category_names = {
    "captacao_ted_total": "TED",
    "captacao_st_total": "ST",
    "captacao_ota_total": "OTA",
    "captacao_rf_total": "RF",
    "captacao_td_total": "TD",
    "captacao_prev_total": "PREV",
}
df_long["Categoria"] = df_long["Categoria"].map(category_names)

# Criar gráfico de barras empilhadas horizontal com escala logarítmica
fig = px.bar(
    df_long,
    x="Valor_Log",  # Usar valor logarítmico para visualização
    y="codigo_assessor",
    color="Categoria",
    title="Decomposição da Captação por Código de Assessor (Top 50)",
    labels={
        "codigo_assessor": "Código do Assessor",
        "Valor_Log": "Captação",
    },
    custom_data=["Valor_Masked", "Categoria"],
    orientation="h",
)

# Atualizar layout para melhor visualização
fig.update_traces(
    hovertemplate="<b>Assessor:</b> %{y}<br>"
    + "<b>Categoria:</b> %{customdata[1]}<br>"
    + "<b>Valor:</b> %{customdata[0]}<br>"
    + "<extra></extra>"
)
fig.update_layout(
    xaxis_title="Captação",
    xaxis=dict(showticklabels=False),  # Esconder valores no eixo X
    yaxis=dict(categoryorder="total ascending", showticklabels=False),
    yaxis_title="",
    plot_bgcolor="white",
    paper_bgcolor="black",
    title_font_color="white",
    height=800,
    barmode="stack",
    showlegend=True,
    legend=dict(
        title="Categoria",
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02,
        font=dict(color="white"),
    ),
)

fig.update_xaxes(title_font=dict(color="white"))

fig.show()


In [7]:
# Criar visualização da relação de clientes com escala logarítmica
df_plot = df.copy()
df_plot["total_clientes"] = df_plot["clientes_pf"] + df_plot["clientes_pj"]
df_plot = df_plot[df_plot["total_clientes"] > 0]
df_plot = df_plot.sort_values(by="total_clientes", ascending=False).head(50)
df_plot["codigo_assessor"] = df_plot["codigo_assessor"].astype(str)

# Selecionar apenas colunas de clientes relevantes
clientes_columns = ["clientes_pf", "clientes_pj"]

# Criar DataFrame no formato longo para plotly
df_long = df_plot.melt(
    id_vars=["codigo_assessor"],
    value_vars=clientes_columns,
    var_name="Categoria",
    value_name="Valor",
)

# Filtrar valores zero para evitar espaços em branco
df_long = df_long[df_long["Valor"] > 0]

# Aplicar transformação logarítmica para melhor visualização de valores pequenos
# Adicionar 1 para evitar log(0)
df_long["Valor_Log"] = df_long["Valor"].apply(
    lambda x: np.log10(x + 1) if x > 0 else 0
)

# Criar coluna mascarada com valores originais
df_long["Valor_Masked"] = df_long["Valor"].apply(mask_value)

# Mapear nomes mais amigáveis
category_names = {
    "clientes_pf": "PESSOA FÍSICA",
    "clientes_pj": "PESSOA JURÍDICA",
}
df_long["Categoria"] = df_long["Categoria"].map(category_names)

# Criar gráfico de barras empilhadas horizontal com escala logarítmica
fig = px.bar(
    df_long,
    x="Valor_Log",  # Usar valor logarítmico para visualização
    y="codigo_assessor",
    color="Categoria",
    title="Relação de Clientes por Código de Assessor (Top 50)",
    labels={
        "codigo_assessor": "Código do Assessor",
        "Valor_Log": "Clientes",
    },
    custom_data=["Valor_Masked", "Categoria"],
    orientation="h",
)

# Atualizar layout para melhor visualização
fig.update_traces(
    hovertemplate="<b>Assessor:</b> %{y}<br>"
    + "<b>Categoria:</b> %{customdata[1]}<br>"
    + "<b>Valor:</b> %{customdata[0]}<br>"
    + "<extra></extra>"
)
fig.update_layout(
    xaxis_title="Clientes",
    xaxis=dict(showticklabels=False),  # Esconder valores no eixo X
    yaxis=dict(categoryorder="total ascending", showticklabels=False),
    yaxis_title="",
    plot_bgcolor="white",
    paper_bgcolor="black",
    title_font_color="white",
    height=800,
    barmode="stack",
    showlegend=True,
    legend=dict(
        title="Categoria",
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02,
        font=dict(color="white"),
    ),
)

fig.update_xaxes(title_font=dict(color="white"))

fig.show()


In [8]:
# Criar visualização da relação de volume operado em ordens com escala logarítmica
df_plot = df.copy()
df_plot["volume_operado_total"] = (
    df_plot["volume_operado_rf"] + df_plot["volume_operado_rv"]
)
df_plot = df_plot[df_plot["volume_operado_total"] > 0]
df_plot = df_plot.sort_values(by="volume_operado_total", ascending=False).head(
    50
)
df_plot["codigo_assessor"] = df_plot["codigo_assessor"].astype(str)

# Selecionar apenas colunas de clientes relevantes
volume_columns = ["volume_operado_rf", "volume_operado_rv"]

# Criar DataFrame no formato longo para plotly
df_long = df_plot.melt(
    id_vars=["codigo_assessor"],
    value_vars=volume_columns,
    var_name="Categoria",
    value_name="Valor",
)

# Filtrar valores zero para evitar espaços em branco
df_long = df_long[df_long["Valor"] > 0]

# Aplicar transformação logarítmica para melhor visualização de valores pequenos
# Adicionar 1 para evitar log(0)
df_long["Valor_Log"] = df_long["Valor"].apply(
    lambda x: np.log10(x + 1) if x > 0 else 0
)

# Criar coluna mascarada com valores originais
df_long["Valor_Masked"] = df_long["Valor"].apply(mask_value)

# Mapear nomes mais amigáveis
category_names = {
    "volume_operado_rf": "RENDA FIXA",
    "volume_operado_rv": "RENDA VARIÁVEL",
}
df_long["Categoria"] = df_long["Categoria"].map(category_names)

# Criar gráfico de barras empilhadas horizontal com escala logarítmica
fig = px.bar(
    df_long,
    x="Valor_Log",  # Usar valor logarítmico para visualização
    y="codigo_assessor",
    color="Categoria",
    title="Volume Operado em Ordens por Código de Assessor (Top 50)",
    labels={
        "codigo_assessor": "Código do Assessor",
        "Valor_Log": "Volume Operado",
    },
    custom_data=["Valor_Masked", "Categoria"],
    orientation="h",
)

# Atualizar layout para melhor visualização
fig.update_traces(
    hovertemplate="<b>Assessor:</b> %{y}<br>"
    + "<b>Categoria:</b> %{customdata[1]}<br>"
    + "<b>Valor:</b> %{customdata[0]}<br>"
    + "<extra></extra>"
)
fig.update_layout(
    xaxis_title="Volume Operado",
    xaxis=dict(showticklabels=False),  # Esconder valores no eixo X
    yaxis=dict(categoryorder="total ascending", showticklabels=False),
    yaxis_title="",
    plot_bgcolor="white",
    paper_bgcolor="black",
    title_font_color="white",
    height=800,
    barmode="stack",
    showlegend=True,
    legend=dict(
        title="Categoria",
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02,
        font=dict(color="white"),
    ),
)

fig.update_xaxes(title_font=dict(color="white"))

fig.show()
